In [1]:
import nibabel as nib
from nibabel.processing import resample_to_output
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import Axes3D
from glob import glob
import re
import os

seed = 42
np.random.seed(seed)  # for reproducibility
import matplotlib.animation as animation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from skimage import measure
from skimage.draw import ellipsoid
import ipyvolume as ipv

In [2]:
data_t1 = np.load("/Dedicated/jmichaelson-sdata/comppsych/lesion/Hackathon_Lesion_Data/t1_np_array.npy")
lables_t1 = np.load("/Dedicated/jmichaelson-sdata/comppsych/lesion/Hackathon_Lesion_Data/t1_labels.npy")

In [3]:
data_array_mask = np.load("/Dedicated/jmichaelson-wdata/mcrichter/HackUiowa2018/DATA/All_subs_405_LM_np_array.npy")
labels_array_mask = np.load("/Dedicated/jmichaelson-wdata/mcrichter/HackUiowa2018/DATA/All_subs_405_labels.npy")

In [4]:
all_masks = data_array_mask.sum(axis=0)
all_masks.shape

(182, 218, 182)

In [51]:
fig = ipv.figure()
masks = ipv.volshow(all_masks.T, data_min =1, level = [.1,.5,.75], downscale = 1, stereo=True)
ipv.pylab.xlim(0, 182)
ipv.pylab.zlim(0, 218)
ipv.pylab.zlim(0, 182)
ipv.style.use('minimal')
# fig.camera_control = 'orbit'
ipv.show()

VkJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdsZXZlbHM6JyksIEZsb2F0U2xpZGVyKHZhbHVlPTAuMSwgbWF4PTEuMCwgc3RlcD0wLjDigKY=


## Create individual frames for gif

In [52]:
def set_view(figure, angle):
    ipv.view(angle)
    ipv.savefig('temp/' + str(angle).zfill(3) + '_temp_im' + '.png')
for angle in range(360):
    set_view(fig, angle)

Output()

## Convert images to gif

In [53]:
import glob
import os
import re
import moviepy.editor as mpy

gif_name = 'all_masks_rotating_z_ipyvolume'
file_list = glob.glob('./temp/*.png') # Get all the pngs in the temp directory
fps = 20

list.sort(file_list, key=lambda x: int(re.findall('\d+', x)[0])) # Sort the images by #, this may need to be tweaked for your use case

clip = mpy.ImageSequenceClip(file_list, fps=fps)
clip.write_gif('{}.gif'.format(gif_name), fps=fps)
clip.write_videofile('{}.mp4'.format(gif_name), fps=fps)


[MoviePy] Building file all_masks_rotating_z_ipyvolume.gif with imageio


100%|██████████| 361/361 [00:47<00:00,  8.07it/s]


[MoviePy] >>>> Building video all_masks_rotating_z_ipyvolume.mp4
[MoviePy] Writing video all_masks_rotating_z_ipyvolume.mp4


100%|██████████| 361/361 [00:03<00:00, 96.41it/s] 


[MoviePy] Done.
[MoviePy] >>>> Video ready: all_masks_rotating_z_ipyvolume.mp4 



In [ ]:
ipv.save("ipv_masks_combined.html")

## Extract brain surface

Up until now ipyvolume doesn't support oppacity on trinagle mesh surfaces :(

In [66]:
verts, faces, normals, values = measure.marching_cubes_lewiner(data_t1[0,:,:,:], 0, step_size = 1)

In [67]:
colors = np.zeros((faces.shape[0] * 2,) + (4,))

In [68]:
colors[:,0] = .7
colors[:,1] = .8
colors[:,2] = 1
colors[:,3] = 0.1

In [69]:
ipv.figure()

x = verts[:,0]
y = verts[:,1]
z = verts[:,2]

mesh = ipv.plot_trisurf(x, y, z, triangles=faces, color=colors)
# and also mark the vertices
# ipv.scatter(x, y, z, marker='sphere', color='blue', size = .1)
# ipv.xyzlim(-2, 2)
ipv.style.use('minimal')
ipv.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


In [70]:
ipv.save("surface_subject_0.html")

In [230]:
def set_view(figure, framenr, fraction):
    ipv.view(fraction*360, -60)
ipv.movie('test.gif', set_view, fps=20, frames=360)

Output()

In [253]:
from ipyvolume.moviemaker import MovieMaker
mm = MovieMaker(stream = fig, camera = fig.camera)
mm.widget_main

VkJveChjaGlsZHJlbj0oVG9nZ2xlQnV0dG9uKHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1JlY29yZCcsIGljb249dSdjaXJjbGUnKSwgRHJvcGRvd24oaW5kZXg9MSwgb3B0aW9ucz0oKCfigKY=


('wrote', 'moviemaker.json')


In [259]:
ipv.view(50)

(50, 3.5083546492674388e-15, 1.9999999999999996)

In [254]:
fig

RmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBwcm9qZWN0aW9uTWF0cml4PSgyLjM1NTg1MjM2NTgyMzc1MjcsIDAuMCzigKY=


In [266]:
import ipywebrtc as webrtc

In [267]:
webrtc.MediaRecorder(stream = fig, filename = "test_webrtc")

AttributeError: 'module' object has no attribute 'MediaRecorder'

In [5]:
%matplotlib notebook
from numpy import *    

def get_cube(mask, s=0, e=182, everything=False):
    if everything:
        cube = mask
    else:
        cube = mask[s:e, s:e, s:e]
    flat = cube.flatten()
    positions = np.where(cube > -1)
    return cube, flat, positions

def cutoff(flat, value):
    return np.array([0 if x < value else x for x in flat])

def norm(a):
    return a / max(a.flatten())

def log_transform(a):
    x = ma.log(a)
    return x.filled(0)

CUTOFF_VALUE = 0



cube,  cube_flat, positions = get_cube(all_masks, everything = True)
cube_cut = cutoff(cube_flat, CUTOFF_VALUE)

non_zeroes = cube_cut != 0

non_zero_positions = np.apply_along_axis(lambda x: x[non_zeroes], 1, positions)
non_zero_cube = cube_cut[non_zeroes]


nz_cube_normalized = norm(non_zero_cube)

rgba_colors = cm.viridis(nz_cube_normalized)
# the fourth column needs to be the alphas
rgba_colors[:, 3] = nz_cube_normalized**5

cube,  cube_flat, pos = get_cube(all_masks, everything = True)

non_zeroes = cube_flat != 0

nz_pos = np.apply_along_axis(lambda x: x[non_zeroes], 1, pos)
nz_cube = cube_flat[non_zeroes]